In [9]:
# import statements for rest of notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
#plt.rc('text', usetex=True)
plt.rc('font', family='serif')
mpl.rcParams.update({'font.size': 20})
mpl.rcParams['figure.figsize'] = 8,8
import math as m   # importing math library outside function definition saves time 
import numpy as np

from IPython.display import Image
import scipy as sci
from scipy.misc import derivative
import scipy.optimize as opt
import sympy as sym
from numpy import pi
#might opt for from numpy import pi to save myself the trouble

In [57]:
#this needs to be cleaned an the ray-material interaction must be included 

class Ray:
    def __init__(self, x, y, angle, wavelength=0):
        
        #vector quantities
        self.sourceLocation = (x,y)
        self.angle = angle
        self.unitVector = [np.cos(angle), np.sin(angle)]
        
        #line quantities
        self.slope = (self.unitVector[1]/self.unitVector[0])
        self.yIntercept = y - x*self.slope # y = mx + b --> b = y - mx
        self.xIntercept = - self.yIntercept/self.slope # x = -b/m
        
        #plot quantities
        self.locationHistory = [self.sourceLocation] # keep track of the prevous interface points
        self.angleHistory = [angle*180/np.pi]
        self.slopeFromRight = 0
        if(self.slope < 0):
            self.slopeFromRight = 1
        
        #light quantities
        self.wavelength = wavelength
        
        #program quantities
        self.objIndex = -1
        self.identity = 42 #this can be set up in the ray generator
        
        
    def update_location_history(self, x, y, angle, objIndex):
        
        #update vector quantities:
        self.sourceLocation = (x,y)
        self.angle = angle
        self.unitVector = [np.cos(angle),np.sin(angle)]
        
        #update line quantities
        self.slope = (self.unitVector[1] / self.unitVector[0])
        self.yIntercept = y - x*self.slope # y = mx + b --> b = y - mx
        self.xIntercept = - self.yIntercept/self.slope # x = -b/m
        
        #update plot quantities
        self.locationHistory.append((x,y))
        self.angleHistory.append(angle*180/np.pi)
        
        #update program quantity
        self.objIndex = objIndex
        
    def plot_ray(self): #this is currently unused
        a = zip(*self.locationHistory)
        xPoints,yPoints = a #this should be equivalent to xPoints, yPoints = a[0], a[1]
        plt.plot(xPoints, yPoints, color='blue')#this probably should reference a figure object so we know where we are plotting
        
    def printString(self):
        return "{0}\n{1}\n{2}\n".format(self.slope, self.xIntercept,np.asarray(self.angleHistory)*180/np.pi)

In [58]:
class RayGenerator:
    def __init__(self, x, y, numberOfRays, typeOf="", x2=0, y2=0, startAngle=0, stopAngle=2*np.pi,startLambda=1, endLambda=1,customRayList=[]):
        self.theRayList = []
        if(typeOf != ""):
            if(typeOf == "point_source"):
                if startAngle == 0 and stopAngle == 2*np.pi:
                    theAngleList = np.linspace(startAngle, stopAngle, numberOfRays+1)
                else:
                    theAngleList = np.linspace(startAngle, stopAngle, numberOfRays)
                for i, angle in enumerate(theAngleList):#still have to put in the color code
                    self.theRayList.append(Ray(x,y,angle, 0))#300+i*(400/numberOfRays)*10**-9))#color code
            elif(typeOf == "beam_source"):
                if (x1-x0) != 0 or (y1-y0) != 0:
                    rayAngle = np.arctan2((x1-x0),-(y1-y0))
                else:
                    raise(Exception("beam width must be non-zero"))
                xList = np.linspace(x0, x1, numberOfRays)
                yList = np.linspace(y0, y1, numberOfRays)
                pointList = list(zip(xList, yList))
                theRayList = []
                for i, point in enumerate(pointList):
                    #for j in range(100):
                    #ray = Ray(point[0],point[1],rayAngle,0)# (400+j*(980/100))*10**-9)
                    self.theRayList.append(Ray(point[0],point[1],rayAngle,0))
                    #flesh this out more so that lambda is evenly spaced between the start and stop
            else:
                raise(Exception("Unknown Source Type, please enter one of the following\nbeam_source or point_source"))
        else:
            self.theRayList=customRayList
        #I don't know if this pass is required but you can put it in for the ability to customize your own ray
        #distribution



In [132]:
#change the vertical line handling so that the second list are the value of the bounding functions
#this amounts to a scaffolding in which each case though be tested in case errors were made

class lens:
    def __init__(self,surfaceList=[],n=lambda x : 1.5):
        
        self.refractiveIndex = n
        self.surfaceList = surfaceList
    
    
    
    #note these class methods must take on an constructor properties we need such as material properties
    #
    # to do:
    # put in the material properties portion
    #
    @classmethod
    def predefined(cls, name, nFunction=lambda x : 1.5):#I should probably include the r1, r2, height, and width values in these examples
        '''
        Here is a series of predefined surfaces that could be used as examples of the behavior of each lens type
        They also provide concrete models of what is generalized in the gaussian optics section below so that 
        someone wishing to use the code could troubleshoot via mimicing the structure shown below
        '''
        if(name == "biconvex"):
            f1 = lambda x : -(-(x-5)**2 + 9)**(1/2) + 5
            f2 = lambda x : (-(x-5)**2 + 9)**(1/2) + 1
            x0 = opt.fsolve(lambda x: f1(x)-f2(x), 2, xtol=1e-10, maxfev=200)
            x1 = opt.fsolve(lambda x: f1(x)-f2(x), 7, xtol=1e-10, maxfev=200)
            return cls(surfaceList = [[[x0[0],x1[0]],[f1,f2]]],n=nFunction)

        elif(name == "biconcave"):
            mySurfaceList = []
            f1 = lambda x : -(5/4 - x/4)**2 + 5
            f2 = lambda x : (x/4 - 5/4)**2 + 6
            mySurfaceList.append([[3],[f1(3),f2(3)]])
            mySurfaceList.append([[3,7],[f1,f2]])
            mySurfaceList.append([[7],[f1(3),f2(3)]])
            return cls(surfaceList=mySurfaceList,n=nFunction)
            

        elif(name == "planar convex"):
            f1 = lambda x : -(-(x-5)**2 + 9)**(1/2) + 5
            f2 = lambda x : 3
            x0 = opt.fsolve(lambda x: f1(x)-f2(x), 2, xtol=1e-10, maxfev=200)
            x1 = opt.fsolve(lambda x: f1(x)-f2(x), 7, xtol=1e-10, maxfev=200)
            return cls(surfaceList = [[[x0[0],x1[0]],[f1,f2]]],n=nFunction)

        elif(name == "planar concave"):
            mySurfaceList = []
            f1 = lambda x : -(5/4 - x/4)**2 + 5
            f2 = lambda x : 6
            mySurfaceList.append([[3],[f1(3),f2(3)]])
            mySurfaceList.append([[3,7],[f1,f2]])
            mySurfaceList.append([[7],[f1(7),f2(7)]])
            return cls(surfaceList=mySurfaceList, n=nFunction)

        elif(name == "meniscus convex"):
            f1 = lambda x: -(-(x-5)**2 + 16)**(1/2) + 6
            f2 = lambda x : -(-(x-5)**2 + 36)**(1/2) + 9
            x0 = opt.fsolve(lambda x: f1(x)-f2(x), 2, xtol=1e-10, maxfev=200)
            x1 = opt.fsolve(lambda x: f1(x)-f2(x), 8, xtol=1e-10, maxfev=200)
            return cls(surfaceList = [[[x0[0],x1[0]],[f1,f2]]],n=nFunction)

        elif(name == "meniscus concave"):#I could save using the append opperation and memory initialization by just expliciting spelling out what I wanted my list to be
            mySurfaceList = []
            f1 = lambda x : -(-(x-5)**2 + 9)**(1/2) + 4.5
            f2 = lambda x : -(-(x-5)**2 + 49)**(1/2) +8
            mySurfaceList.append([[3],[f1(3),f2(3)]])
            mySurfaceList.append([[3,7],[f1,f2]])
            mySurfaceList.append([[7],[f1(7),f2(7)]])
            return cls(surfaceList = mySurfaceList,n=nFunction)
        else:
            raise(Exception("Unknown lens type, please enter one of the following:\nbiconvex, biconcave, planar convex, planar concave, meniscus convex, meniscus concave"))
    
    @classmethod
    def gaussianOptics(cls,r1=0,r2=0,width=0,height=0,nFunction=lambda x:1.5):
        '''
        The basic stucture of this function is to deal with the various gaussian optics inputs we could recieve and handle them appropriately
        After some error handling most cases take on the following characteristic structure:
        
        0. for concave: ensure enough information is provided
        1. determine rmax for plotting and doing math later
        2. define the two surface functions
        3. for convex: determine where the surfaces intersect
        4. use the helper method to concisely construct our surface data 
    
        '''
        def abrvFunc(r1,r2,f1,f2,height,x0=0,x1=0): #this is a helper function to make the code more readable(or less eye watering if you prefer)
            if(x0 != 0 or x1 != 0):
                if(x1[0]-x0[0] <= height):#we can either raise an exception or just use the maximal diameter
                    raise(Exception("this height exceeds the limitations imposed by the specified diamter"))
            xmin = rmax - height/2
            xmax = rmax + height/2
            mySurfaceList = [[[xmin],[f1(xmin),f2(xmax)]]]
            mySurfaceList.append([[xmin,xmax],[f1,f2]])
            mySurfaceList.append([[xmax],[f1(xmax),f2(xmax)]])
            return mySurfaceList
        
        if(((r1 or r2 or height) and width) or ((r1 or r2) and height) or (r1 and r2)):#this should say that at least two of the following have to be given
            #deal with negative width
            #widths should change as the relative orientation of the radii change
            if(width < 0 or height < 0):
                raise(Exception("width and height are constrained to be greater than or equal to zero"))
            elif(width == 0 and height == 0 ):
                raise(Exception("at minimum a width or height must be specified for a lens along with r1 or r2"))
            elif(r1 > 0 and r2 > 0 and r1 < r2):#meniscus convex
                if(not width):
                    width = abs(r1-(r1**2 -height**2/4)**(1/2) - r2 + (r2**2-height**2/4)**(1/2))
                rmax = r2
                f1 = lambda x : (-(x-rmax)**2 + r1**2)**(1/2) + rmax+1
                f2 = lambda x : (-(x-rmax)**2 + r2**2)**(1/2) + rmax+1 - width + r2 - r1
                x0 = opt.fsolve(lambda x: f1(x)-f2(x), 0, xtol=1e-10, maxfev=200)
                x1 = opt.fsolve(lambda x: f1(x)-f2(x), 2*rmax, xtol=1e-10, maxfev=200)
                if(height):
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height,x0,x1),n=nFunction)
                return cls(surfaceList = [[x0[0],x1[0]],[f1,f2]],n=nFunction)
                
                    
            elif(r1 > 0 and r2 == 0):#plano-convex case 1
                if(not width):
                    width = abs(r1-(r1**2-height**2/4)**(1/2))
                rmax = r1
                f1 = lambda x : (-(x-rmax)**2 + r1**2)**(1/2) + rmax+1
                f2 = lambda x : rmax+1+width
                x0 = opt.fsolve(lambda x: f1(x)-f2(x), 0, xtol=1e-10, maxfev=200)
                x1 = opt.fsolve(lambda x: f1(x)-f2(x), 2*rmax, xtol=1e-10, maxfev=200)
                if(height):    
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height,x0,x1),n=nFunction)
                return cls(surfaceList = [[x0[0],x1[0]],[f1,f2]],n=nFunction)

            elif(r1 > 0 and r2 < 0):#biconvex
                r2 = -r2
                if(not width):
                    width = abs(r1-(r1**2-width**2/4)**(1/2)) + abs(r2-(r2**2-width**2/4)**(1/2))
                rmax = max(r1, r2)
                f1 = lambda x : -(-(x-rmax)**2 + r1**2)**(1/2) + rmax+1
                f2 = lambda x : (-(x-rmax)**2 + r2**2)**(1/2) + rmax+1 + width-r1-r2
                x0 = opt.fsolve(lambda x: f1(x)-f2(x), 0, xtol=1e-10, maxfev=200)
                x1 = opt.fsolve(lambda x: f1(x)-f2(x), 2*rmax, xtol=1e-10, maxfev=200)
                if(height):
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height,x0,x1),n=nFunction)
                return cls(surfaceList = [[x0[0],x1[0]],[f1,f2]],n=nFunction)

            elif(r1 == 0 and r2 > 0):#plano-convex case 2
                if(not width):
                    width = abs(r2-(r2**2-height**2/4)**(1/2))
                rmax = r2
                f1 = lambda x : (-(x-rmax)**2 + r2**2)**(1/2) + width+1
                f2 = lambda x : 1
                x0 = opt.fsolve(lambda x: f1(x)-f2(x), 0, xtol=1e-10, maxfev=200)
                x1 = opt.fsolve(lambda x: f1(x)-f2(x), 2*rmax, xtol=1e-10, maxfev=200)
                if(height):
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height,x0,x1),n=nFunction)
                return cls(surfaceList = [[x0[0],x1[0]],[f1,f2]],n=nFunction)

            elif(r1 == 0 and r2 == 0):#rectangle
                mySurfaceList = [[[1],[1,height+1]]]
                mySurfaceList.append([[1,width+1],[lambda x : 1, lambda x : height+1]])
                mySurfaceList.append([[width+1],[1,height+1]])
                return cls(surfaceList=mySurfaceList,n=nFunction)

            elif(r1 == 0 and r2 < 0):#plano-concave case 1
                if(height == 0 or width == 0):
                    raise(Exception("Insufficient information provided for concave class object"))
                r2 = -r2
                rmax = r2
                f1 = lambda x : 1
                f2 = lambda x : -(-(x-rmax)**2 + r2**2)**(1/2) + 1+rmax+width
                return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height),n=nFunction) 
            
            elif(r1 < 0 and r2 > 0):#biconcave
                if(height == 0 or width == 0):
                    raise(Exception("Insufficient information provided for concave class object"))
                r1 = -r1
                rmax = max(r1,r2)
                f1 = lambda x : (-(x-rmax)**2 + r1**2)**(1/2) + rmax+1
                f2 = lambda x : -(-(x-rmax)**2 + r2**2)**(1/2) + rmax+1 + width +r1+r2
                return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height),n=nFunction)

            elif(r1 < 0 and r2 == 0):#plano-concave case 2
                if(height == 0 or width == 0):
                    raise(Exception("Insufficient information provided for concave class object"))
                r1 = -r1
                rmax = r1
                f1 = lambda x : (-(x-rmax)**2 + r1**2)**(1/2) + 1
                f2 = lambda x : 1 + r1 + width
                return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height),n=nFunction)
            
            elif(r1 > 0 and r2 > 0):#meniscus concave
                if(height == 0 or width == 0):
                    raise(Exception("Insufficient information provided for concave class object"))
                rmin = min(r1,r2)
                rmax = max(r1,r2)
                f1 = lambda x : (-(x-rmax)**2 + r1**2)**(1/2) + rmax+1
                f2 = lambda x : (-(x-rmax)**2 + r2**2)**(1/2) + rmax+1 + width + r2 - r1
                if(height < 2*rmin):#reasonable height
                    return cls(surfaceList=abrvFunc(r1,r2,f1,f2,height),n=nFunction)
                else:#unreasonable height
                    raise(Exception("This is an unreasonable height for a meniscus concave lens"))
                    #self.surfaceList.append([rmax-rmin],[])
                    #self.surfaceList.append([[rmax-rmin,rmax+rmin],[f1,f2]])#this is the we'll fix it approach
                    #self.surfaceList.append([[rmax+rmin],[]])
        else:
            raise(Exception("Not enough information supplied or height and width = 0 which is impossible"))#probably should print inputs
        
    #unsure how much burden I want to put on the user
    #we could take a dictionary of inputs which might be more readable but that might also make the whole process more complicated
    #insofar as translating that dictionary set up into our list structure
    @classmethod
    def custom(cls,mySurfaceList,nFunction=lambda x:1.5):
        return cls(surfaceList=mySurfaceList,n=nFunction)
    #there should probably some rhobust error checking for dealing with the defined surfaces
    
    def sellmeier(self,wavelength):
        B1,B2,B3,C1,C2,C3 = self.sellmeier
        if(B1 and B2 and B3 and C1 and C2 and C3):
            wavelength *= 1e6
            return lambda x : (1 + B1*wavelength**2/(wavelength**2 - C1) + B2*wavelength**2/(wavelength**2 - C2) + B3*wavelength**2/(wavelength**2 - C3) )**(1/2)
        return self.refractiveIndex

In [133]:
#[([xStart,xStop],[func1, func2]),...]

In [146]:
class driver:
    def __init__(self,n,listOfRays,lens):
        self.nMedium = n
        self.listOfRays = listOfRays
        self.lens = lens
        
    def rayFunc(self):
        
        def normalVectorFunc(function, value): # Value is the x value where the given ray intersects the lens surface
            fprime = derivative(function, value, dx=1e-13, order=21)
            return [-fprime/(1+fprime**2)**(0.5),1/(1+fprime**2)**(0.5)]
        
        def thetaCritical(n1, n2):#these should be effective indices in the case where they are functions
            return np.arcsin(n2/n1)
        
        #expected inputs for line: [slope, x intercept], surface is a function, value is a reasonable estimate for where the zero occurs
        def rayIntersectSurface(line, surface, value): 
            return opt.fsolve(lambda x: surface(x)-x*line[0] + line[0]*line[1], value, xtol=1e-9, maxfev=200)#f(x) - m(x-x0) = 0
        
        
        passes = 0
        listOfRays = self.listOfRays
        listOfSurfaces = self.lens.surfaceList 
        #print()
        while(passes < 3):
            for i in range(len(listOfRays)):#idea: why keep track of all of the possible points of intersection when we only care about the min?
                ray = listOfRays[i]
                minDistance = 1e10
                minDistSurface = -1
                xMin, yMin = -1,-1
                x0,y0 = ray.sourceLocation
                for s in range(len(listOfSurfaces)):
                    domain, surfaces = listOfSurfaces[s]
                    #print(domain,surfaces)
                    #should add the shifted case, where there are two line segments 
                    if(len(domain) == 1):#this is a vertical line 
                        pointOfIntersection = ray.slope*domain[0] + ray.yIntercept
                        if(pointOfIntersection >= surfaces[0] and pointOfIntersection <= surfaces[1]):
                            #calculate the distance traveled
                            distance = ((domain[0]-x0)**2 + (pointOfIntersection-y0)**2)**(1/2)
                            if(distance < minDistance and distance > 0.001):
                                minDistance = distance
                                xMin, yMin = domain, pointOfIntersection
                                minDistSurface = [domain[0],surfaces[:2]] #I use this as the label to differentiate between the vertical line case and the rest
                        elif(len(surfaces) == 4):
                            if(pointOfIntersection >= surfaces[2] and pointOfIntersection <= surfaces[3]):
                                distance = ((domain[0]-x0)**2 + (pointOfIntersection-y0)**2)**(1/2)
                                if(distance < minDistance):
                                    minDistance = distance
                                    xMin, yMin = domain, pointOfIntersection
                                    minDistSurface = [domain[0],surfaces[2:4]]
                    #should probably include error checking to ensure that the length of any of these domain lists is either 1 or 2 
                    else:#we're not working with a vertical line
                        #given a domain (a,b), my strategy is to use fsolve with an intial guess of (a+b)/2
                        #unsure if this is sufficiently robust
                        initial_guess = (domain[0]+domain[1])/2
                        #
                        #ATTENTION:
                        #this adjustment in the lower limit of the range is a quick fix and should be addressed more throughly
                        #
                        #
                        for i in range(passes,len(surfaces)):#we have to loop over all the surfaces defined on this domain
                            surface_i = surfaces[i]
                            listOfPointsOfIntersection = rayIntersectSurface([ray.slope, ray.xIntercept], surface_i, initial_guess)
                            #I think the best point is going to be the first one on the list, this may be wrong
                            a, b = domain
                            pointOfIntersection = listOfPointsOfIntersection[0]
                            if(pointOfIntersection > a and pointOfIntersection < b):
                                #we've got a valid intersection
                                y1 = surface_i(pointOfIntersection)
                                distance = ((pointOfIntersection-x0)**2 + (y1-y0)**2)**(1/2)
                                if(distance < minDistance):#put in a min distance, this might fix the ray not moving problem
                                    minDistance = distance
                                    xMin, yMin = pointOfIntersection, y1
                                    minDistSurface = [surface_i]
                                    
                #we have now found the next point of intersection or exhausted all possibilities so we can move on to the rest of this
                if(minDistSurface == -1):#no points of intersection, may as well move on to the next ray
                    continue
                #before we proceed we again need to have the special case handled for vertical lines :(
                if(len(minDistSurface) == 2): #the special case
                    #which way does our normal vector point?
                    #unpack our list
                    #print("I'm reached")
                    x1, yList = minDistSurface
                    if(x0 < x1):
                        normalVector = [1, 0]
                    else:
                        normalVector = [-1,0]
                    #this probably needs to consider the vertical line in the interior of the lens
                else:
                    normalVector = normalVectorFunc(minDistSurface[0],xMin)
                
                theta1 = np.arccos(np.dot(ray.unitVector, normalVector))
                #we need to determine whether we're inside or outside the lens
                if(ray.objIndex == -1):
                    n1 = self.nMedium
                    n2 = self.lens.refractiveIndex
                else:
                    n1 = self.lens.refractiveIndex#this could be generalized fairly easily by having the objIndex index the list of our lenses
                    n2 = self.nMedium
                wavelength = ray.wavelength
                if(theta1 >= thetaCritical(n1(wavelength),n2(wavelength))):#internal reflection case need to do this still
                    continue
                theta2 = np.arcsin((n1(wavelength)/n2(wavelength))*np.sin(theta1))
                xN, yN = normalVector
                refractedRay1 = [-xN*np.cos(-theta2) + yN*np.sin(-theta2), \
                         -xN*np.sin(-theta2) - yN*np.cos(-theta2)]
                refractedRay2 = [-xN*np.cos(theta2) + yN*np.sin(theta2), \
                         -xN*np.sin(theta2) - yN*np.cos(theta2)]
                if(np.dot(refractedRay1, ray.unitVector) < np.dot(refractedRay2, ray.unitVector)):
                    refractedRay = refractedRay1
                else:
                    refractedRay = refractedRay2
                ray.angleHistory.append(theta1)
                ray.angleHistory.append(theta2)
                refractedXaxisTheta = np.arctan2(refractedRay[1],refractedRay[0])#new theta of refracted ray relative to x axis
                if(refractedXaxisTheta < 0):#gotta stay in the "right" quadrant
                    refractedXaxisTheta += np.pi
                refractedRayIntercept = xMin - refractedRay[0]/refractedRay[1]*yMin#new x intercept
                if(n2 == self.nMedium):
                    objI = 0
                else:
                    objI = 1
                ray.update_location_history(xMin,yMin,refractedXaxisTheta,objI)
            passes+=1
        #massive sadness ahead as we do the plots
    def plotter(self,fig):
        sub = fig.add_subplot(111)
        surfaceList = self.lens.surfaceList
        #first we have to handle graphing the lens or lenses
        for s in range(len(surfaceList)):
            domain, surfaces = surfaceList[s]
            if(len(domain) == 1):#this will need some refining for the 4 point case
                x = domain[0]
                xList = [x]*len(surfaces)
                sub.plot(xList,surfaces)
            else:
                x = np.linspace(domain[0],domain[1],1000)
                for function in surfaces:#this will probably need special handling for constant functions
                    if(function(1) == function(2) and function(1) == function(1.5)):#this seems to be a constant function
                        yList = [function(1)]*len(x)
                    else:
                        yList = function(x)
                    sub.plot(x, yList)
        rayList = self.listOfRays
        for i in range(len(rayList)):
            xAx, yAx = zip(*rayList[i].locationHistory)
            xAxList, yAxList = list(xAx), list(yAx)
            #print(xAxList,yAxList)
            length = len(xAx)
            if(rayList[i].slopeFromRight == 0):
                if(rayList[i].slope < 0):
                    xAxList.append(xAxList[length-1]-5)
                    yAxList.append(yAxList[length-1]-5*rayList[i].slope)
                else:
                    xAxList.append(xAxList[length-1]+5)
                    yAxList.append(yAxList[length-1]+5*rayList[i].slope)
            else:
                if(rayList[i].slope > 0):
                    xAxList.append(xAxList[length-1]+5)
                    yAxList.append(yAxList[length-1]+5*rayList[i].slope)
                else:
                    xAxList.append(xAxList[length-1]-5)
                    yAxList.append(yAxList[length-1]-5*rayList[i].slope)
            sub.plot(xAxList,yAxList, 'm')

        #plt.gca().invert_yaxis()
        #plt.show()
            

bug list:
1. problem with getting the surface finding feature to "jump" to the next surface, i.e. not just have the ray stand still
2. problem with refracting off of the flat sides in a concave lens
3. there probably should be explicit handling for vertical incoming rays
4. the meniscus concave predefined doesn't seem to work, unsure why
    removing the passes patch seemed to make some progress


In [151]:
#tester cell
#5,-1,10, np.pi/3, 2*np.pi/3
myRayList = RayGenerator(5,-1,20,"point_source",startAngle=np.pi/3,stopAngle=2*np.pi/3)
myLens = lens.predefined("meniscus concave",lambda x: 1.8)
theDriver = driver(lambda x:1,myRayList.theRayList,myLens)
theDriver.rayFunc()
fig = plt.figure()
theDriver.plotter(fig)
plt.ylim(-1,10)
plt.show()
#for ray in theDriver.listOfRays:
#    print(ray.angleHistory)

C:\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in arcsin


In [ ]:
#to figure out the surface intersection bit try extending the surfaces past where the functions are limited to, this is not a 
#permenate solution but is some sort of stop gap measure

C:\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:11: RuntimeWarning: invalid value encountered in arcsin
C:\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: invalid value encountered in sqrt
C:\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:15: RuntimeWarning: invalid value encountered in sqrt


In [12]:
#for i in b:
#    print(i.angleHistory)

In [1]:
myList = [1,2,3]

In [2]:
g,h,j = myList

In [3]:
print(g)

1


In [26]:
a = [[1,2],[3,4,5]]
b,c = a

In [27]:
print(b,c)

[1, 2] [3, 4, 5]


In [96]:
np.arccos(np.dot([(2)**(-1/2),2**(-1/2)],[1,0]))*180/np.pi

45.0